In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import datasets
from src import config

In [ ]:
hyper_parameters = config.HyperParameters()

dataset: datasets.Dataset = datasets.load_dataset(
    hyper_parameters.data_config.dataset, split="train"
)  # type: ignore

README.md:   0%|          | 0.00/583 [00:00<?, ?B/s]

data/train-00000-of-00010-45de7542ea7caa(…):   0%|          | 0.00/492M [00:00<?, ?B/s]

data/train-00001-of-00010-941dfd963f3fb7(…):   0%|          | 0.00/487M [00:00<?, ?B/s]

data/train-00002-of-00010-584b44bfff9f35(…):   0%|          | 0.00/495M [00:00<?, ?B/s]

data/train-00003-of-00010-c4cd69058db789(…):   0%|          | 0.00/492M [00:00<?, ?B/s]

...

data/train-00006-of-00010-d6594693b4c0e5(…):   0%|          | 0.00/490M [00:00<?, ?B/s]

data/train-00007-of-00010-e2c13ff1928c45(…):   0%|          | 0.00/491M [00:00<?, ?B/s]

data/train-00008-of-00010-dbfd724640b186(…):   0%|          | 0.00/487M [00:00<?, ?B/s]

data/train-00009-of-00010-06eda47bab140d(…):   0%|          | 0.00/488M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/30000 [00:00<?, ? examples/s]

In [5]:
dataset[:3]

{'image': [<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x428>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x426>,
  <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>],
 'caption': ['A group of zebras grazing in the grass.',
  'a number of people standing around a large group of luggage bags',
  'A yellow commuter train traveling past some houses.']}

In [6]:
type(dataset[:3])

dict

In [7]:
from torch.utils.data import DataLoader

In [17]:
from PIL import Image
from typing import Any


def collate_fn(batch: list[dict[str, Image.Image | str]]) -> dict[str, list[list[dict[str, Any]]]]:
    images = [
        [
            {
                "role": "user",
                "content": [
                    {
                        "type": "image",
                        "image": data_instance["image"],
                    },
                ],
            }
        ]
        for data_instance in batch
    ]
    texts = [
        [
            {
                "role": "user",
                "content": [{"type": "text", "text": data_instance["caption"]}],
            }
        ]
        for data_instance in batch
    ]
    return {
        "images": images,
        "texts": texts,
    }


dl = DataLoader(
    dataset=dataset,
    batch_size=16,
    shuffle=True,
    pin_memory=True,
    drop_last=True,
    collate_fn=collate_fn,
)

In [18]:
batch = next(iter(dl))

/Users/sachinthaka/personal_work/clip_jepa/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [22]:
from torch.utils.data import Dataset

isinstance(dataset, datasets.Dataset)

True

In [21]:
type(dataset)

datasets.arrow_dataset.Dataset

In [19]:
batch

{'images': [[{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>}]}],
  [{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x427>}]}],
  [{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x428>}]}],
  [{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x427>}]}],
  [{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=550x365>}]}],
  [{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x427>}]}],
  [{'role': 'user',
    'content': [{'type': 'image',
      'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=640x480>}]}],
  [{'role': 'user',
    

In [16]:
from src import data, config, model

import torch

In [2]:
hyper_parameters = config.HyperParameters()
train_dl, valid_dl = data.get_dataset(hyper_parameters)

2025-10-05 18:09:39.375 | INFO     | src.data:get_dataset:50 - Loading Dataset...
Using the latest cached version of the dataset since sayakpaul/coco-30-val-2014 couldn't be found on the Hugging Face Hub
Found the latest cached dataset configuration 'default' at /Users/sachinthaka/.cache/huggingface/datasets/sayakpaul___coco-30-val-2014/default/0.0.0/abdde3200f533ddae5bed2438057f1f7ea2d5131 (last modified on Sun Oct  5 12:15:53 2025).
2025-10-05 18:09:43.975 | INFO     | src.data:get_dataset:56 - Splitting dataset
2025-10-05 18:09:43.978 | INFO     | src.data:_get_dataloaders:25 - Creating Dataloaders


In [3]:
batch = next(iter(train_dl))

/Users/sachinthaka/personal_work/clip_jepa/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [4]:
len(batch["images"])

8

In [17]:
jepa_model = model.CLIPJepaModel(hyper_parameters.llm_model_config, device=torch.device("mps"))

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
batch.keys()

dict_keys(['images', 'texts'])

In [20]:
with torch.no_grad():
    text_embeddings = jepa_model.embed_text(batch["texts"])
    # image_embeddings = jepa_model.embed_image(batch["images"])

In [ ]:
(
    text_embeddings.shape,
    image_embeddings.shape,
    (text_embeddings**2).sum(),
    (image_embeddings**2).sum(),
)

(torch.Size([8, 2048]),
 torch.Size([8, 2048]),
 tensor(8., device='mps:0'),
 tensor(8., device='mps:0'))